# Intro to HPC: Solar Cell project

Project: Solar Power for Affordable Housing through Computational Design of Low-Cost/High-Efficiency Solar Cells.
Author: AlvaroVM [https://alvarovm.github.io](http://alvarovm.github.io)

Version: 0.0.2

## Part 4  Feaure exploraration, Molecular Maps: PCA using RDKIT, part 2

Author: AlvaroVM [https://alvarovm.github.io](http://alvarovm.github.io)
Version: 0.0.1

For this example we define in SMILES string two groups of molecules with different substituents, such as -CH3, -O, -F, -Cl, and- I , in molecules with six carbons 1) in a ring and 2) in chain. Those molecules would be added to a list, additionally we add a 'certain' property , this could be used later as a flag.

In [ ]:
import sys
import os
SRC_DIR='..'

In [ ]:
sys.path.append(os.path.join(SRC_DIR, 'code'))
import utils

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
#https://github.com/jmcarpenter2/swifter
#import swifter
#2-TSNE-UMAP-map-cuda-Copy1

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import DataStructs 
from rdkit.Chem import Draw
from rdkit.Chem.rdMolDescriptors import  GetHashedMorganFingerprint
from rdkit.DataStructs import ConvertToNumpyArray

from sklearn.manifold import TSNE

import hdbscan

utils.plot_settings2()

results_path = os.path.join(SRC_DIR,'results')

In [ ]:
df = pd.read_pickle('../data/../data/extended_db_Zindo_Nov_2019_V5_cannfp_clust_lem.pkl').fillna(value = 0)
print('Column names: {}'.format(str(df.columns.tolist())))
print('Table Shape: {}'.format(df.shape))
#df.head(2)

### Exercises

* Clean the tables removing the rows with zeros in `lambda_tddft (nm)` and `lambda_sTDA (nm)`

* Compare `lambda_tddft (nm)` vs `lambda_sTDA (nm)`, use  t-SNE clusters to colour the points. Is there any relationship? 

* Compare `lambda_tddft (nm)` vs `lambda_sTDA (nm)` in the most populated clusters. Are there clusters that correlate better?

* Compute the difference between `lambda_tddft (nm)` and `lambda_sTDA (nm)`. Analyze the differences by cluster.

* Repeat the same analysis with the ZINDO method